In [ ]:
# importing packages
import sys
import random
import torch  
import gym
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
from collections import namedtuple, deque
import wandb
from plyer import notification

In [ ]:
# Constants
GAMMA = 0.99
# learning_rate = 3e-4

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class NoBaselinePolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, learning_rate=3e-4):
        super(NoBaselinePolicyNetwork, self).__init__()

        self.num_actions = num_actions
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, num_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.softmax(self.linear2(x), dim=1)
        return x 
    
    def get_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.forward(Variable(state))
        highest_prob_action = np.random.choice(self.num_actions, p=np.squeeze(probs.detach().numpy()))
        log_prob = torch.log(probs.squeeze(0)[highest_prob_action])
        return highest_prob_action, log_prob

In [ ]:
def compute_returns(rewards):
    discounted_rewards = []
    for t in range(len(rewards)):
        Gt = 0 
        pw = 0
        for r in rewards[t:]:
            Gt = Gt + GAMMA**pw * r
            pw = pw + 1
        discounted_rewards.append(Gt)
    return discounted_rewards

def update_policy(policy_network, rewards, log_probs):
    discounted_rewards = compute_returns(rewards)
    discounted_rewards = torch.tensor(discounted_rewards)
    policy_gradient = []
    for log_prob, Gt in zip(log_probs, discounted_rewards):
        policy_gradient.append(-log_prob * Gt)
    policy_network.optimizer.zero_grad()
    policy_gradient = torch.stack(policy_gradient).sum()
    policy_gradient.backward()
    policy_network.optimizer.step()

In [ ]:
def train(environment, render=False, hidden_size=128, learning_rate=3e-4, max_episode_num=750, max_steps=1000):
    env = gym.make(environment["name"])
    env.seed(random.randint(0,100))
    policy_net = NoBaselinePolicyNetwork(env.observation_space.shape[0], env.action_space.n, hidden_size, learning_rate).to(device)
    reward_sat = False
    all_rewards = []
    scores_window = deque(maxlen=100)
    for episode in range(max_episode_num):
        state = env.reset()
        log_probs = []
        rewards = []
        episode_reward = 0
        for steps in range(max_steps):
            if render:
                env.render()
            action, log_prob = policy_net.get_action(state)
            new_state, reward, done, _ = env.step(action)
            episode_reward+=reward
            log_probs.append(log_prob)
            rewards.append(reward)
            if done:
                scores_window.append(episode_reward)
                if not reward_sat:
                    update_policy(policy_net, rewards, log_probs)
                all_rewards.append(episode_reward)
                break
            state = new_state

        if episode % 100 == 0:
            print('\rEpisode {}\tTotal Reward: {:.2f}\tAverage Reward: {:.2f}'.format(episode, episode_reward, np.mean(scores_window)))
        if np.mean(scores_window) >= environment["avg_reward_threshold"]:
            print('\nEnvironment solved in {:d} episodes!\tAverage Reward: {:.2f}'.format(episode, np.mean(scores_window)))
            reward_sat=True
    
    env.close()
    return all_rewards

In [ ]:
def run_agent(environment, learning_rates):
    all_scores = []
    for lr in learning_rates:
        five_runs=[]
        for _ in range(5):
            print("Run ", _, ": ", lr)
            scores = train(environment, False, 128, lr)
            five_runs.append(scores)
        all_scores.append(five_runs)
    notification.notify(title="Run Complete",
                        message="Your rewards have been plotted")
    return all_scores

In [ ]:
# environment = {"name": 'Acrobot-v1', "avg_reward_threshold": -100}
# # learning_rates = np.linspace(2e-3, 9e-3, 15)
# learning_rates = [0.003]
# all_scores_1 = run_agent(environment, learning_rates)

In [ ]:
environment = {"name": 'CartPole-v1', "avg_reward_threshold": 475}
# learning_rates = np.linspace(2e-3, 9e-3, 15)
learning_rates = [0.003283]
all_scores_2 = run_agent(environment, learning_rates)

In [ ]:
# np.save("./plots/acrobot_without_baseline", np.array(all_scores_1))
# np.save("./plots/cartpole_without_baseline", np.array(all_scores_2))